# Read and visualize dicom files

### Imports

In [4]:
import sys
import os
from glob import glob
import napari
import pydicom
import zarr
import numpy as np
import skimage

### Paths

In [5]:
img_dir = '/Users/nikhil/Downloads/Series-2/'

### Read dicoms 

In [6]:
# load the DICOM files
files = []
print('glob: {}'.format('{}'.format(img_dir)))
for fname in glob('{}/*.dcm'.format(img_dir)):
#     print("loading: {}".format(fname))
    files.append(pydicom.dcmread(fname))

print("file count: {}".format(len(files)))

# skip files with no SliceLocation (eg scout views)
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))

# ensure they are in the correct order
slices = sorted(slices, key=lambda s: s.SliceLocation)

# pixel aspects, assuming all slices are the same
ps = slices[0].PixelSpacing
ss = slices[0].SliceThickness
ax_aspect = ps[1]/ps[0]
sag_aspect = ps[1]/ss
cor_aspect = ss/ps[0]

# create 3D array
img_shape = list(slices[0].pixel_array.shape)
img_shape.append(len(slices))
img3d = np.zeros(img_shape)

# fill 3D array with the images from the files
for i, s in enumerate(slices):
    img2d = s.pixel_array
    img3d[:, :, i] = img2d

glob: /Users/nikhil/Downloads/Series-2/
loading: /Users/nikhil/Downloads/Series-2/Image-27.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-33.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-32.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-26.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-30.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-24.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-18.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-19.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-25.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-31.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-35.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-21.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-20.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-34.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-22.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-36.dcm
loading: /Users/nikhil/Downloads/Series-2/Image-37.dcm
loading: /Users/nikhil/Do

### View dicoms

In [8]:
with napari.gui_qt():
    viewer = napari.view_image(img3d)